In [8]:
import re
import os
import pandas as pd
import ngrams
import numpy as np
import math
import csv
import fasttext

In [16]:
#全局变量
sourceAttributeList = []
#token的字典，用来记录token出现的次数nt
tokenDict={}

In [11]:
#将属性名按照标点，分割成token
def snake_case_split(line):
    #Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
    #r表示非转义的原始字符串
    #由于正则表达式通常都包含反斜杠，所以你最好使用原始字符串来表示它们。模式元素(如 r'\t'，等价于 '\\t')匹配相应的特殊字符。
    #[...]用来表示一组字符,单独列出：[amk] 匹配 'a'，'m'或'k'
    line_split = re.split(r'[\s_]',line.strip())
    line_split = [line.strip() for line in line_split if len(line.strip())>0]
    return line_split




#将属性按照大写字母，分割成token
#"firstName"分成 ['First', 'Name']
def camel_case_split(str):
    if "a"<=str[0]<="z" :
        strTemp=str[0].upper()+str[1:]
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', strTemp)
    else :
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)




#将属性名先按照snakeCase拆分，再按照camelCase拆分
def attributeToToken(str):
    tokenList = []
    for snakeToken in snake_case_split(str):
        for camelToken in camel_case_split(snakeToken):
            camelToken = camelToken.lower()
            corpusTokenList = ngrams.segment2(camelToken) [1]
            tokenList = tokenList + corpusTokenList
    return tokenList

In [13]:
a = attributeToToken("myheartwillgoonandon")
a

['my', 'heart', 'will', 'go', 'on', 'and', 'on']

In [14]:
class Attribute:
    def __init__(self, attributeName, originateFrom):
        self.attributeName = attributeName
        self.tokenList = attributeToToken(attributeName)
        #属性的词嵌入表示
        #加权平均
        #权和N有关
        #numOfAttribute得读完整个数据湖才知道
        #self.wordEmbedding = tokenToVector(self.tokenList)
        
        #属性来自哪张表
        self.originateFrom = originateFrom

In [ ]:
class Table:
    def __init__(self, ):

In [17]:
#getTokenWeight函数用到numOfAttribute
numOfAttribute = 0
numOfTable = 0

#将湖中所有属性读到attributeList中
#for循环中的info代表文件名
for info in os.listdir('/usr/dataLake/datasets/BenchmarkCsvfiles'): 
    #去掉.csv后缀
    tableName = info[0:-4]
    domain = os.path.abspath('/usr/dataLake/datasets/BenchmarkCsvfiles') #获取文件夹的路径
    info = os.path.join(domain,info) #将路径与文件名结合起来就是每个文件的完整路径
    print(info)
    df = pd.read_csv(info)
    for attributeName in list(df):
        #Attribute类的构造函数会更新tokenDict
        x=Attribute(attributeName, tableName)
        numOfAttribute = numOfAttribute+1
        #通过append函数将自定义类的对象放入list中
        sourceAttributeList.append(x)
        #print(sourceAttributeList[i])可证明是Attribute类的对象
    
    numOfTable = numOfTable + 1
    if numOfTable == 10:
        break

/usr/dataLake/datasets/BenchmarkCsvfiles/t_93f3d6f7fc6aa6ff____c13_0____0.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_70941cace7dd1c45____c12_1____1.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_e9efd5cda78af711____c4_1____1.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_ca85e8f9eef5b9d5____c5_1____0.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_dc9442ed0b52d69c____c13_0____4.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/PED_SK_DTL_SNF____c6_0____4.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_93f3d6f7fc6aa6ff____c21_0____4.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/CTA_Ridership_Avg_Weekday_Bus_Stop_Boardings_in_October_2012____c7_0____2.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_014c2259cce4ab13____c25_0____4.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/CTA_Ridership_Avg_Weekday_Bus_Stop_Boardings_in_October_2012____c2_1____3.csv


In [18]:
#修改完毕
for sourceAttribute in sourceAttributeList:
    for token in sourceAttribute.tokenList:
        if token in tokenDict:
              tokenDict[token] = tokenDict[token] + 1
        else:
              tokenDict[token] = 1

In [19]:
print(tokenDict)

{'address': 2, 'auth': 2, 'description': 4, 'granted': 2, 'date': 9, 'lic': 2, 'no': 5, 'pub': 2, 'text': 2, 'reg': 7, 'service': 6, 'number': 2, 'type': 9, 'other': 2, 'details': 3, 'tao': 2, 'covered': 2, 'by': 2, 'area': 2, 'effective': 2, 'end': 2, 'trading': 2, 'name': 4, 'quality': 25, 'indicator': 25, 'indicateurs': 29, 'de': 29, 'qualit': 29, 'kind': 1, 'pmc': 1, 'he': 1, 'pm': 4, 'te': 4, 's': 3, 'grou': 1, 'colour': 1, 'consis': 1, 'pls': 1, 'mottle': 1, 'frq': 1, 'root': 2, 'freq': 1, 'sec': 1, 'grad': 1, 'rest': 1, 'ri': 1, 'soil': 1, 'group': 1, 'exp': 1, 'op': 1, 'registration': 1, 'status': 1, 'subsidies': 1, 'finish': 1, 'point': 2, 'received': 1, 'start': 1, 'via': 1, 'a': 1, 'lightings': 1, 'boardings': 1, 'cross': 1, 'street': 3, 'location': 2, 'on': 2, 'routes': 1, 'stop': 2, 'id': 2, 'aircraft': 6, 'make': 4, 'airport': 1, 'icao': 1, 'incident': 1, 'occ': 2, 'ocean': 1, 'operator': 2, 'organization': 5, 'country': 5}


In [20]:
#加载模型
fasttext.FastText.eprint = lambda x: None
model = fasttext.load_model("/usr/dataLake/result/fil9.bin")

def getTokenWeight(token):
    nt = tokenDict[token]
    #默认以e为底
    return math.log(numOfAttribute/nt)

#归一化向量
def normalize(x):
    return x/np.linalg.norm(x)

#将属性名的tokenList转换为词嵌入向量
#获取token对应的词嵌入向量，归一化。加权平均后，再归一化一下。
def tokenToVector(tokenList):
    #获取词嵌入
    wordEmbedding = model.get_word_vector(tokenList[0])
    #函数返回值
    unitVector = getTokenWeight(tokenList[0]) * normalize(wordEmbedding)
    #加权平均的分母
    totalWeight = getTokenWeight(tokenList[0])
    for token in tokenList[1:]:
        wordEmbedding = model.get_word_vector(token)
        unitVector = unitVector + getTokenWeight(token) * normalize(wordEmbedding)
        totalWeight = totalWeight+getTokenWeight(token)    
    return normalize(unitVector/totalWeight)


In [21]:
#计算attributeVector，并输出到文件中

with open('attributeVector.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    for sourceAttribute in sourceAttributeList:
        sourceAttribute.wordEmbedding = tokenToVector(sourceAttribute.tokenList)
        writer.writerow(sourceAttribute.wordEmbedding)

In [24]:

for i in range(16,25):
    print(sourceAttributeList[i].attributeName)

Quality Indicator
Indicateurs de qualit�
Quality Indicator
Indicateurs de qualit�.2
Quality Indicator
Indicateurs de qualit�.3
Quality Indicator
Indicateurs de qualit�.4
Quality Indicator
Indicateurs de qualit�.5
Quality Indicator
Indicateurs de qualit�.6
Quality Indicator
Indicateurs de qualit�.7
Quality Indicator
Indicateurs de qualit�.8
Quality Indicator
Indicateurs de qualit�.9


In [31]:
#用list存储列的类别信息
labels=[]
with open('/usr/dataLake/DRD/ball-k-means/labels.csv', 'r') as f:
    for line in f:
        labels.append(int(line))

int

In [43]:
#已经检查通过 20221129   kdzkdz
def read_csv(file_name):
    f = open(file_name, 'r')
    content = f.read()
    final_list = list()
    rows = content.split('\n')
    #-1是为了删除末尾的空列表
    for row in rows[0:-1]:
        str_list = row.split(',')
        #将字符串转换为int
        num_list = [int(i) for i in str_list]        
        final_list.append(num_list)
    return final_list

#二维list，每个元素是int
#数据正确性已经验证
clusterPointIndex = read_csv("/usr/dataLake/DRD/ball-k-means/ClusterPointIndex.csv")
print(clusterPointIndex)
#已经检查通过 20221129   kdzkdz

[[69, 71], [13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 34, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46], [9, 66], [81, 82, 83, 84, 85, 86], [96, 97, 98, 99, 100], [58], [26, 33], [78, 106], [2, 56, 61, 70], [6, 62], [79], [11, 68], [91, 92], [10, 67], [32, 50], [76], [75], [3, 57], [73], [77, 90], [30], [14, 15, 35, 36], [80, 107], [7, 8, 63, 64, 94, 95], [25, 74], [89], [93], [27, 47, 48, 49], [0, 1, 53, 54], [28, 52], [51], [12, 72], [55], [101, 102, 103, 104, 105], [87], [4, 59], [29, 31], [5, 60], [88], [65]]


In [ ]:
k=40
#键 是 表名，值 为 列表
CandidateTablePair={}

#循环40个簇
for cluster in range(0, 40 ):
    for i in range(0 , len(clusterPointIndex[cluster])-1 ):
        for j in range(i+1, len(clusterPointIndex[cluster]) ):
            CandidateTablePair

###todo：！！！

In [ ]:
#输入target表
#target表作为增量，放到程序的最后
targetTable='t_93f3d6f7fc6aa6ff____c13_0____0'
targetAttributeList = []
info = targetTable+'.csv'
domain = os.path.abspath('/usr/dataLake/datasets/BenchmarkCsvfiles') #获取文件夹的路径
info = os.path.join(domain,info) #将路径与文件名结合起来就是每个文件的完整路径
df = pd.read_csv(info)
for targetAttributeName in list(df):
    #targetAttribute的token被额外加了一次
    targetAttribute=Attribute(targetAttributeName, targetTable)
    targetAttributeList.append(targetAttribute)

### todo：有新表应该更新tokenDict